# Exercise 05. Advanced Topic: Serving your RLlib Model using Ray Serve and adding an in-game item recommendation system.

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives
In this this tutorial, you will learn:

 * [How to deploy a trained policy into production using Ray Serve](#ray_serve)

In [1]:
# import required packages

import time
import gym
from gym.spaces import Discrete, MultiDiscrete

import ray
from ray import serve

print(f"gym: {gym.__version__}")
print(f"ray: {ray.__version__}")

# !ale-import-roms --import-from-pkg atari_py.atari_roms

gym: 0.21.0
ray: 3.0.0.dev0


### Using Ray Serve to deploy a trained policy into production <a class="anchor" id="ray_serve"></a>

In [ ]:
serve.start()

@serve.deployment(route_prefix="/in-game-recommendations")
class ServeModel:
    def __init__(self, checkpoint_path) -> None:
        self.algo = CRRTrainer(
            config=offline_rl_config,
        )
        self.algo.restore(checkpoint_path)

    async def __call__(self, request: Request):
        json_input = await request.json()
        obs = json_input["observation"]
        # Translate obs back to np.arrays.
        np_obs = OrderedDict(tree.map_structure(lambda s: np.array(s) if isinstance(s, list) else s, obs))
        action = self.algo.compute_single_action(np_obs, explore=False)
        return {"action": action}

ServeModel.deploy(checkpoint_file)

In [ ]:

# Request 5 actions of an episode from served policy.
obs = env.reset()

for _ in range(5):
    # Convert numpy arrays to lists (needed for transfer).
    obs = tree.map_structure(lambda s: s.tolist() if isinstance(s, np.ndarray) else s, obs)

    print(f"-> Sending observation {obs}")
    resp = requests.get(
        "http://localhost:8000/in-game-recommendations", json={"observation": obs}
    )
    response_json = resp.json()
    print(f"<- got {response_json}")
    obs, _, _, _ = offline_rl_env.step(np.array(response_json["action"]))

### Exercises

1. 

### Homework

1. 

### References

* 